### 5 mrt
Uitgezocht welke tool we moeten gebruiken om quality controle uit te voeren op de minion data. Fastqc schijnt niet goed te werken op minion data, dus we gaan Nanoplot gebruiken. deze is te installeren via conda: `conda install -c bioconda nanoplot`

!To do:
- data verkennen
- inlezen nanoplot

### 11 mrt
Martijn heeft uitgelegd wat de bedoeling nu is. We gaan alle drie een andere tool (ik zoek kraken2 uit) gebruiken om het bicrobioom te bepalen en de resultaten daarvan vervolgens met elkaar vergelijken.

## 12 mrt
kraen2 installeren in conda: `conda install -c bioconda kraken2`
database uitzoeken: \
[16S_Greengenes: Archaea and Bacteria](https://pmc.ncbi.nlm.nih.gov/articles/PMC1489311/) \
[16S silvia: Bacteria, Archaea and Eukarya](https://www.arb-silva.de/)

16S lijkt uitgebreider, dus deze gaan we gebruiken.

commando: \
kraken2 : start kraken2 \
--db /data/datasets/16S_Silva : de database die we gebruiken \
--threads 8 : threads \
--use-names : gebruikt de wetenschappelijke namen ipv taxids(cijfers)
--report kraken2_report.txt : samenvatting rapport \
--output kraken2_output.txt : de output per read \
/commons/Themas/Thema07/metagenomics/zoutstress/16S/.. : input (seq data)

de input moet via een loop ingevuld worden: \
de mappen zijn: \
zoet_20_30cm_rep1  zoet_20_30cm_rep2  zout_20_30cm_rep1  zout_20_30cm_rep2 \
met daarin: \ 
FAY08769_pass_barcode07_85e26567_1cef8301_N.fastq.gz (waarin N een int is)

Maar omdat ze overlappingen hebben moeten ze in aparte mappen blijven.

Dus per map uitvoeren. acc_list.txt maken daarvoor.

Voor wgs gaan we een apart commando maken. \
[database](https://benlangmead.github.io/aws-indexes/k2) uitzoeken voor wgs:
- k2_core_nt : Very large collection, inclusive of GenBank, RefSeq, TPA and PDB
- k2_pluspf : Standard plus Refeq protozoa & fungi
- k2_pluspfp : Standard plus Refeq protozoa, fungi & plant
- k2_standard : Refeq archaea, bacteria, viral, plasmid, human1, UniVec_Core

Aangezien ze allemaal al op het systeem staan (ivm opslag ruimte) gebruiken we de meest volledige: k2_pluspfp.

### 13 mrt
Met behulp van Ramon mappen probleem opgelost: \
`find /students/2024-2025/Thema07/metagenomics/zoutstress/testing_data/ -type d -wholename '*rep*' |` \
Omdat de mappen in bestanden zijn veranderd, moeten we nu `-type f -name` gebruiken ipv `-type d -wholename`.


### 14 mrt
[Kraken2](https://software.cqls.oregonstate.edu/updates/docs/kraken2/MANUAL.html) wil op een andere manier de input bestanden krijgen: \
Dus voor 16S: \
`kraken2 --db /data/datasets/16S_Silva --threads 8  --use-names --report kraken2_report.txt --output kraken2_16S_output.txt /students/2024-2025/Thema07/metagenomics/zoutstress/data/16S/*rep*.fastq`

En voor WGS: \
`kraken2 --db /data/datasets/k2_pluspfp --threads 8  --use-names --report kraken2_report.txt --output kraken2_WGS_output.txt /students/2024-2025/Thema07/metagenomics/zoutstress/data/WGS/*rep*.fastq`

Nu moeten we er alleen nog voor zorgen dat de bestanden gescheiden blijven. \
Dus voor 16S: \
`find /students/2024-2025/Thema07/metagenomics/zoutstress/data/16S/ -name '*rep*.fastq' | \
    parallel 'kraken2 --db /data/datasets/KRAKEN2_INDEX/16S_Silva --threads 8 --use-names \
    --report /homes/ieineke/metagenomicssalinitystress/kraken2_16S/kraken2_report/kraken2_report_{/.}.txt \
    --output /homes/ieineke/metagenomicssalinitystress/kraken2_16S/kraken2_output/kraken2_output_{/.}.txt {}'`

En voor WGS: \
`find /students/2024-2025/Thema07/metagenomics/zoutstress/data/WGS/ -name '*rep*.fastq' | \
    parallel 'kraken2 --db /data/datasets/KRAKEN2_INDEX/k2_pluspfp --threads 8 --use-names \
    --report /homes/ieineke/metagenomicssalinitystress/kraken2_WGS/kraken2_report/kraken2_report_{/.}.txt \
    --output /homes/ieineke/metagenomicssalinitystress/kraken2_WGS/kraken2_output/kraken2_output_{/.}.txt {}'`

`{/.}` : neemt de bestandsnaam zonder het pad \
`{}` : hier worden alle door `find` gevonden bestanden ingevuld als input


ze zijn beide uitgevoerd.

#### BRacken
Eerst installeren: `conda install bioconda::bracken` \

`find /students/2024-2025/Thema07/metagenomics/zoutstress/outputs/kraken2/kraken2_16S/kraken2_report -name 'kraken2_report_*.txt' | \
    parallel 'bracken -d /data/datasets/KRAKEN2_INDEX/16S_Silva -i {} \
    -o /students/2024-2025/Thema07/metagenomics/zoutstress/outputs/bracken_op_kraken2/16S_bracken/bracken_output_{/.}.txt \
    -r 730 -l species -t 10'`

`find /students/2024-2025/Thema07/metagenomics/zoutstress/outputs/kraken2/kraken2_WGS/kraken2_report -name 'kraken2_report_*.txt' | \
    parallel 'bracken -d /data/datasets/KRAKEN2_INDEX/k2_pluspfp -i {} \
    -o /students/2024-2025/Thema07/metagenomics/zoutstress/outputs/bracken_op_kraken2/WGS_bracken/bracken_output_{/.}.txt \
    -r 760 -l species -t 10'`


Deze mogelijk later veranderen (r: reads, l:classification level, t: threshold): \
- -r 730 voor 16S en 760 voor WGS is gekozen op basis van de gemiddelde reads lengthes uit de nanorapporten \
(16S gem length reads: 736, 732.6, 630.4, 736.6)(WGS gem lenght reads: 1070.2, 761.7, 1002.8, 1108.0)
- -l species is gekozen, omdat dit logisch is voor onderzoeksvraag: wat is het microbioom.
- -t 10 is gekozen, omdat minion foutgevoelig is en een hoge threshold voorkomt (hopelijk) onbetrouwbare resulaten.

Een foutmelding geeft aan dat er eerst een kmer distribution file gemaakt moet worden met bracken-build: \
hiervoor is het handiger om een symlink te proberen te gebruiken, aangezien ik dan de output bij de database in de map kan plaatsten zonder dat medestudenten uit andere groepen hier last van hebben:
voor 16S: `ln -s /data/datasets/KRAKEN2_INDEX/16S_Silva /students/2024-2025/Thema07/metagenomics/zoutstress/outputs/kraken2/kraken_db_symlink` \
voor WGS: `ln -s /data/datasets/KRAKEN2_INDEX/k2_pluspfp /students/2024-2025/Thema07/metagenomics/zoutstress/outputs/kraken2/kraken_db_symlink`

dan nu bracken:
voor 16S: `bracken-build -d /students/2024-2025/Thema07/metagenomics/zoutstress/outputs/kraken2/kraken_db_symlink/16S_Silva -l 730` \
voor WGS: `bracken-build -d /students/2024-2025/Thema07/metagenomics/zoutstress/outputs/kraken2/kraken_db_symlink/k2_pluspfp -l 760`

We moeten de -l aanpassen, omdat we als hoogste kmer-bestand 250 reads hebben. niet ideaal, maar moet maar: \
bracken-build -d /students/2024-2025/Thema07/metagenomics/zoutstress/outputs/kraken2/kraken_db_symlink/16S_Silva/database250mers.kmer_distrib -l 250

FOUTMELDING, lukt me niet om deze op te lossen: \
***ERROR: Database library /students/2024-2025/Thema07/metagenomics/zoutstress/outputs/kraken2/kraken_db_symlink/16S_Silva/library does not exist***

### 18 mrt
Vandaag is opgezocht welke minimum qualiry score voor de minion data moet worden ingesteld voor de trimming. \
Na [onderzoek](https://academic.oup.com/bioinformatics/article/35/3/523/5057155?login=false) blijkt dat 7 een goede waarde is die nog voldoende data en voldoende nauwkerigheid behoudt.


Ook vroeg Ramon 
